In [2]:
using FunctionalCollections
using Iterators
using Pipe
using Compat

@everywhere macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [27]:
@everywhere using JLD
@everywhere data = load("results/data/brown_glove50d.jld")
test_set = data["corpus"]

42004-element Array{Array{ASCIIString,1},1}:
 ASCIIString["``","only","a","relative","handful","of","such","reports","was","received"  …  "of","voters","and","the","size","of","this","city","''","."]                       
 ASCIIString["it","recommended","that","fulton","legislators","act","``","to","have","these"  …  "to","the","end","of","modernizing","and","improving","them","''","."]          
 ASCIIString["the","grand","jury","commented","on","a","number","of","other","topics"  …  "inure","to","the","best","interest","of","both","governments","''","."]               
 ASCIIString["merger","proposed"]                                                                                                                                                
 ASCIIString["however",",","the","jury","said","it","believes","``","these","two"  …  "greater","efficiency","and","reduce","the","cost","of","administration","''","."]         
 ASCIIString["the","city","purchasing","department",",","the","ju

In [4]:
@everywhere function lookup_sowe(data, sent::AbstractString)
    lookup_sowe(data, sent |> split)
end

@everywhere function lookup_sowe{S<:AbstractString}(data, sent::Vector{S})
    sum([data["LL"][:,data["word_indexes"][word]] for word in sent]) 
end

@everywhere function lookup_words(data, path)
    ASCIIString[data["indexed_words"][ii] for ii in path]
end

@everywhere function lookup_indexes{S<:AbstractString}(data, sent::Vector{S})
    Int[data["word_indexes"][word] for word in sent]
end

In [5]:
S = hcat(map(sent->lookup_sowe(data,sent), test_set[1:5])...)

300x5 Array{Float64,2}:
  -4.74163    -2.57656    -5.25187   -0.29441     -2.30494  
   1.36376    -1.06473     3.5523    -0.47571      3.91438  
   2.9576     -1.37417     1.08203   -0.211661    -1.55368  
  -7.45742    -1.5113     -6.52253    0.15203     -3.66229  
   0.501547    0.181163    2.25553    0.391033     2.33487  
   2.36419     0.914427    1.47449   -0.22831     -0.135271 
  -3.05212     0.156627   -1.89791    0.93516     -0.125447 
   0.227302    1.47765     5.56737    0.3722       4.35596  
  -1.3487      1.69361     0.729621  -0.484771     0.131528 
 -57.1041    -37.7398    -59.9178    -2.8967     -45.4103   
   3.78744     1.08485     2.80916   -0.226627     4.48032  
   0.720431   -1.14156     1.29291   -0.0844309   -0.943215 
  -6.63207    -3.55058    -4.19841   -0.37577     -4.75805  
   ⋮                                                        
   2.4757      0.279908    3.64195   -0.121485    -0.0583882
   0.448004   -2.13483    -2.10817   -0.014715    -1.65544  


In [6]:
V = data["LL"];

In [7]:
nWords = size(V,2)
nSents = size(S,2)

5

In [8]:
using NonNegLeastSquares

In [12]:
k=2
C_k = nonneg_lsq(V, S[:,k];alg=:fnnls, max_iter=10)

40482x1 Array{Float64,2}:
 0.0    
 0.0    
 0.0    
 0.0    
 2.35392
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 ⋮      
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    
 0.0    

In [52]:
@pipe lookup_words(data, reduce(append!,Int64[],  [[j for _ in 1:cj] for (j,cj) in enumerate(round(C_k))])) |> sort |> join(_, " ")

"`` `` age by commentators county drainage fate fda improving interior its law law parliament revised season to to various vehemently we we"

In [53]:
test_set[1] |> join 

37-element Array{ASCIIString,1}:
 "``"      
 "only"    
 "a"       
 "relative"
 "handful" 
 "of"      
 "such"    
 "reports" 
 "was"     
 "received"
 "''"      
 ","       
 "the"     
 ⋮         
 "the"     
 "number"  
 "of"      
 "voters"  
 "and"     
 "the"     
 "size"    
 "of"      
 "this"    
 "city"    
 "''"      
 "."       

In [35]:
repeat([1],[1,1,2])

LoadError: LoadError: MethodError: `repeat` has no method matching repeat(::Array{Int64,1}, ::Array{Int64,1})
Closest candidates are:
  repeat{T}(::Array{T,N})
while loading In[35], in expression starting on line 1

In [29]:
lookup_words(data, round(C_k[find(C_k>0)]))

LoadError: LoadError: MethodError: `isless` has no method matching isless(::Int64, ::Array{Float64,2})
Closest candidates are:
  isless(::Real, !Matched::AbstractFloat)
  isless(::Real, !Matched::Real)
  isless(::Integer, !Matched::Char)
while loading In[29], in expression starting on line 1

In [54]:
@pipe test_set[2] |> join(_, " ")

"it recommended that fulton legislators act `` to have these laws studied and revised to the end of modernizing and improving them '' ."

In [9]:
?nonneg_lsq

search: 

**x = nonneg_lsq(A, B; ...)**

Computes the (k-by-n) matrix X with that minimizes vecnorm(A*X - B) subject to X >= 0, where A is an (m-by-k) matrix and B is a (m-by-n) matrix.

## Optional arguments

**alg:** a symbol specifying the algorithm to be used

```
:pivot - Block-pivoting active-set-like method (Kim & Park, 2011)
:fnnls - Fast NNLS, (Bro & De Jong, 1997)
:nnls - Classic active-set method (Lawson & Hanson, 1974)
:convex - Interface to Convex.jl solvers
```

**variant:** a symbol specifying the variant, if applicable, 

**tol:** tolerance for nonnegativity constraints

**max_iter:** maximum number of iterations before function gives up


nonneg_lsq

